In [14]:
import pandas as pd
from sqlalchemy import create_engine
import json
from web3 import Web3
from tqdm.auto import tqdm
from itertools import permutations
import warnings

warnings.filterwarnings('ignore')

# ------------------------------------------------------------------------------
# PASSO 1: Carregar e Preparar os Dados
# ------------------------------------------------------------------------------
print("--- Passo 1: Carregando dados da Mainnet do seu servidor ---")
SERVER_IP = "34.79.99.180" 
db_connection_str = f'postgresql://admin:supersecret@{SERVER_IP}:5433/mempool_data'
df = pd.DataFrame()

try:
    db_engine = create_engine(db_connection_str)
    # Vamos pegar uma amostra maior para aumentar a chance de encontrar ciclos
    df_raw = pd.read_sql("SELECT hash, \"inputData\", to_address, gas_price, base_fee_per_gas FROM transactions", con=db_engine)
    df = df_raw.sample(n=min(5000, len(df_raw)), random_state=42).copy()
    print(f"Sucesso! Carregamos uma amostra de {len(df)} transações para análise.")
    df.dropna(subset=['inputData'], inplace=True)
except Exception as e:
    print(f"--- ERRO ao carregar dados: {e} ---")

# ------------------------------------------------------------------------------
# PASSO 2: Decodificar Dados e Construir Mapa de Preços
# ------------------------------------------------------------------------------
if not df.empty:
    print("\n--- Passo 2: Decodificando transações e construindo mapa de preços...")
    
    # ABI para decodificar swaps e extrair o path
    router_abi_str = '[{"name":"swapExactTokensForTokens","inputs":[{"type":"uint256","name":"amountIn"},{"type":"uint256","name":"amountOutMin"},{"type":"address[]","name":"path"}],"type":"function"}]'
    router_abi = json.loads(router_abi_str)
    generic_contract = Web3().eth.contract(abi=router_abi)

    def extract_path(input_data):
        try:
            input_bytes = bytes.fromhex(input_data[2:])
            func_obj, func_params = generic_contract.decode_function_input(input_bytes)
            # Retornamos apenas o caminho dos tokens
            return func_params.get('path', [])
        except Exception:
            return []

    df['path'] = df['inputData'].apply(extract_path)
    
    # Filtra apenas transações que são swaps diretos (caminho com 2 tokens)
    swaps_df = df[df['path'].apply(lambda x: len(x) == 2)].copy()
    
    # Criamos um "mapa de preços" ou, mais precisamente, um mapa de conexões
    # A chave é um par de tokens (token_in, token_out) e o valor é o endereço do roteador
    price_map = {}
    for index, row in swaps_df.iterrows():
        token_in, token_out = row['path'][0], row['path'][1]
        router_address = row['to_address']
        if (token_in, token_out) not in price_map:
            price_map[(token_in, token_out)] = []
        if router_address not in price_map[(token_in, token_out)]:
            price_map[(token_in, token_out)].append(router_address)
            
    print(f"Mapa de {len(price_map)} pares de troca únicos construído.")

# ------------------------------------------------------------------------------
# PASSO 3: Caçar Ciclos Triangulares
# ------------------------------------------------------------------------------
if 'price_map' in locals():
    print("\n--- Passo 3: Caçando ciclos triangulares (A->B->C->A) ---")
    
    all_tokens = pd.unique(swaps_df['path'].explode())
    found_cycles = []

    # Geramos todas as permutações de 3 tokens
    for p in tqdm(permutations(all_tokens, 3), total=len(all_tokens)**3):
        token_a, token_b, token_c = p[0], p[1], p[2]
        
        # Verifica se os 3 "lados" do triângulo existem no nosso mapa de preços
        if (token_a, token_b) in price_map and \
           (token_b, token_c) in price_map and \
           (token_c, token_a) in price_map:
            
            cycle = f"{token_a} -> {token_b} -> {token_c} -> {token_a}"
            if cycle not in found_cycles:
                found_cycles.append(cycle)
                
    if found_cycles:
        print(f"\nSucesso! Encontrados {len(found_cycles)} potenciais ciclos de arbitragem triangular para investigar:")
        # Mostra os 5 primeiros encontrados
        for cycle in found_cycles[:5]:
            print(f"  - Ciclo Encontrado: {cycle}")
    else:
        print("\nNenhum ciclo triangular completo encontrado nesta amostra de dados.")

else:
    print("Nenhum dado de swap para analisar.")

--- Passo 1: Carregando dados da Mainnet do seu servidor ---
Sucesso! Carregamos uma amostra de 1313 transações para análise.

--- Passo 2: Decodificando transações e construindo mapa de preços...
Mapa de 0 pares de troca únicos construído.

--- Passo 3: Caçando ciclos triangulares (A->B->C->A) ---


0it [00:00, ?it/s]


Nenhum ciclo triangular completo encontrado nesta amostra de dados.
